In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
match_sum = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/STATS_S02.xlsx', 
                         sheet_name='Match_Summary')

In [3]:
matches_list1 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_01')
matches_list2 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_02')

matches_list1['match_id'] = matches_list1['num'].apply(lambda x: 'M00' + str(x) if x<=9 else 'M0' + str(x))
matches_list2['match_id'] = matches_list2['num'].apply(lambda x: 'S02M00' + str(x) if x<=9 else 'S02M0' + str(x))

matches_list = pd.concat([matches_list1[['num','Spin','match_id']],matches_list2[['num','Spin','match_id']]],
                         axis=0)



matches_list.columns
#matches_list = matches_list[['Date','Team One','Team Two']]

Index(['num', 'Spin', 'match_id'], dtype='object')

matches_list

In [4]:
#df_sim = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats//df_all_round_sim.csv')
df_sim2 = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')

#df_sim = pd.concat([df_sim,df_sim2], axis=0).reset_index(drop=True)
df_sim = df_sim2

/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df_01 = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats/df_all_round_sim.csv')
df_curr = pd.DataFrame(columns=df_01.columns.tolist())
try:
    df_curr = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')
    #df_curr = df_curr[df_01.columns.tolist()]
except FileNotFoundError:
    print("df_curr file not found, proceeding with df_01 only.")

df_all = pd.concat([df_01,df_curr], axis=0)
df_all.reset_index(drop=True, inplace=True)


df_all['phase'] = np.where(df_all['legal_balls_bowled']<=36, 'pp', 
                        np.where(df_all['legal_balls_bowled']>=90, 'death',
                            'middle'))

df_sim = df_all

([1, 2, 6, 1, 6, 1, 2, 4, 3, 4, 3, 4, 3, 4, 3, 5, 2, 5, 2, 5],
 [1, 5, 1, 6, 5, 1, 6, 3, 4, 6, 3, 6, 3, 4, 3, 2, 5, 2, 5, 2])

df_sim[df_sim['batting_team']=='CSK']['striker'].unique()

In [6]:
play_style_df = pd.read_csv("/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/play_style.csv")


In [7]:
df_sim.loc[df_sim.isWicket==1, 'partnership_runs'] = np.nan

df_sim['partnership_runs'] = df_sim['partnership_runs'].fillna(method='ffill')
df_sim.loc[df_sim.isWicket==1, 'partnership_runs'] = df_sim.loc[df_sim.isWicket==1, 'partnership_runs']+\
                                            df_sim.loc[df_sim.isWicket==1, 'total_runs']

In [8]:
#samp = df_sim[df_sim.match_id=='M072']

df_sim['before_ball_partnership'] = 0
df_sim['after_ball_partnership'] = 0
df_sim['before_balls'] = 0
df_sim['after_balls'] = 0

for m_id in df_sim.match_id.unique():
    #print(m_id)
    bp = 0
    ap = 0
    bb = 0
    ab = 0
    for inn in df_sim[df_sim.match_id==m_id].innings.unique():
        bp = 0
        ap = 0
        bb = 0
        ab = 0
        for index,row in df_sim[df_sim.match_id==m_id][df_sim.innings==inn].iterrows():
            ap = bp + row['total_runs']
            ab = bb + row['is_faced_by_batter']
            #print(ap,bp)
            df_sim.at[index, 'before_ball_partnership'] = bp
            df_sim.at[index, 'after_ball_partnership'] = ap
            df_sim.at[index, 'before_balls'] = bb
            df_sim.at[index, 'after_balls'] = ab
            bp = ap
            bb = ab
            if row['isWicket']==1:
                bp = 0
                ap = 0
                bb = 0
                ab = 0
        

p_sub = df_sim[['wickets_down','after_ball_partnership']]\
                .sort_values(['wickets_down','after_ball_partnership'], ascending=[True,False])

p_df = p_sub.groupby('wickets_down')['after_ball_partnership'].head(5).reset_index()

#p_df

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/1875421427.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for index,row in df_sim[df_sim.match_id==m_id][df_sim.innings==inn].iterrows():


In [9]:
df_sim['over_1'] = df_sim['legal_balls_bowled']//6
df_sim['over_2'] = df_sim['legal_balls_bowled']%6
df_sim['over_1'] = df_sim['over_1'].astype(str)
df_sim['over_2'] = df_sim['over_2'].astype(str)

df_sim['over'] = df_sim['over_1'] + "." +  df_sim['over_2']
df_sim['over'] = df_sim['over'].astype(float)

df_sim.drop(['over_1','over_2'],axis=1,inplace=True)
df_sim['over_'] = np.ceil(df_sim['over'])
df_sim['over_'] = df_sim['over_'].astype(int)

df_sim['phase'] = np.where(df_sim['over']<=6, "PP", np.where(df_sim['over']<=15, "Middle", "Death"))

In [10]:
df_sim = df_sim.merge(matches_list[['match_id','Spin']], on='match_id', how='left')
df_sim['pitch'] = df_sim['Spin'].apply(lambda x: 'pace' if x<=40 
                                       else 'spin' if x>=60
                                      else 'neutralish')

In [11]:
df_sim = df_sim.merge(play_style_df, on='bowler',
                       how='left')

## spin-pace stats for each team

In [727]:
spin_bat = df_sim[df_sim['bowl_style_parent']=='Spin'].groupby('batting_team').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

spin_bat['SR'] = 100*spin_bat['runs']/spin_bat['balls']
spin_bat['Bat_avg'] = spin_bat['runs']/spin_bat['outs']
spin_bat['bpb'] = spin_bat['balls']/(spin_bat['fours']+spin_bat['sixes'])
spin_bat['dot_%'] = 100*spin_bat['dots']/spin_bat['balls']

pace_bat = df_sim[df_sim['bowl_style_parent']=='Pace'].groupby('batting_team').agg(
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter','sum'),
    outs = ('isBowlerWicket','sum'),
    fours = ('isFour','sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum')
).reset_index()

pace_bat['SR'] = 100*pace_bat['runs']/pace_bat['balls']
pace_bat['Bat_avg'] = pace_bat['runs']/pace_bat['outs']
pace_bat['bpb'] = pace_bat['balls']/(pace_bat['fours']+pace_bat['sixes'])
pace_bat['dot_%'] = 100*pace_bat['dots']/pace_bat['balls']
pace_bat['bowling_type'] = 'pace'
pace_bat['pace_index_int'] = pace_bat['SR']+np.where(pace_bat['Bat_avg']<1000, pace_bat['Bat_avg'], \
                                pace_bat['runs'])- 10*np.where(pace_bat['bpb']<1000, pace_bat['bpb'], \
                                        pace_bat['balls'])-pace_bat['dot_%']

pace_bat['pace_idx'] = np.where(
    ((pace_bat['balls'] < 300) ), #& (pace_bat['balls'] >= 50) 
    pace_bat['pace_index_int'] * ((pace_bat['balls'] / 300) ** 0.5),\
    np.where(pace_bat['balls'] >= 300, 
    pace_bat['pace_index_int'], \
            np.nan)
)


##
spin_bat['spin_index_int'] = spin_bat['SR']+np.where(spin_bat['Bat_avg']<1000, spin_bat['Bat_avg'], \
                                spin_bat['runs'])- 10*np.where(spin_bat['bpb']<1000, spin_bat['bpb'], \
                                        spin_bat['balls'])-spin_bat['dot_%']
spin_bat['bowling_type'] = 'spin'
spin_bat['spin_idx'] = np.where(
    ((spin_bat['balls'] < 300) ),  #& (spin_bat['balls'] >= 50) 
    spin_bat['spin_index_int'] * ((spin_bat['balls'] / 300) ** 0.5),\
    np.where(spin_bat['balls'] >= 300, 
    spin_bat['spin_index_int'], \
            np.nan)
)

##

k1 = 0.02
k2 = 0.1
c1 = 70
c2 = 97

def sigmoid(x):
    if x<=70:
        value = 0.5 + 1 / (1 + np.exp(-k1 * (x-c1)))
    else:
        value = 1 + 1/ (1 + np.exp(-k2 * (x-c2 )))
        
    return value

pace_bat['pace_index'] = pace_bat['pace_idx'].apply(sigmoid)
spin_bat['spin_index'] = spin_bat['spin_idx'].apply(sigmoid)

##
pace_bat.drop(['pace_index_int','pace_idx'], axis=1, inplace=True)
spin_bat.drop(['spin_index_int','spin_idx'], axis=1, inplace=True)



pl_st_df = pd.concat([pace_bat,spin_bat], axis=0)
pl_st_df = pl_st_df.round(2)

In [731]:
pl_st_df[pl_st_df.bowling_type=='spin']

,batting_team,runs,balls,outs,fours,sixes,dots,SR,Bat_avg,bpb,dot_%,bowling_type,pace_index,spin_index
0,CSK,1836,1361,66,159,66,484,134.90,27.82,6.05,35.56,spin,NaN,0.98
1,DC,2646,1764,78,186,125,560,150.00,33.92,5.67,31.75,spin,NaN,1.46
2,GT,2165,1422,51,163,87,410,152.25,42.45,5.69,28.83,spin,NaN,1.77
3,KKR,1629,1120,47,129,68,371,145.45,34.66,5.69,33.12,spin,NaN,1.33
4,LSG,2499,1714,77,190,113,594,145.80,32.45,5.66,34.66,spin,NaN,1.27
5,MI,1983,1363,61,164,68,415,145.49,32.51,5.88,30.45,spin,NaN,1.31
6,PBKS,2545,1770,90,193,114,594,143.79,28.28,5.77,33.56,spin,NaN,1.17
7,RCB,1910,1378,63,144,82,465,138.61,30.32,6.10,33.74,spin,NaN,1.09
8,RR,2670,1761,83,222,108,524,151.62,32.17,5.34,29.76,spin,NaN,1.59
9,SRH,3091,2081,79,232,148,657,148.53,39.13,5.48,31.57,spin,NaN,1.61


## wkt-wise partnership stats

In [93]:
def safe_subtract(a, b):
    if pd.isna(b):
        return np.nan
    if pd.isna(a):
        return np.nan
    return b - a

In [94]:
p_df_all = pd.DataFrame()

teams = ['CSK','DC','GT','LSG','KKR','MI','PBKS','RCB','RR','SRH']

for team in teams:
    wkt = 10


    partnership_df = pd.DataFrame(np.nan, index=range(14), columns= 
                                           ['team','Match','1st Wkt','2nd Wkt','3rd Wkt','4th Wkt','5th Wkt',
                                          '6th Wkt','7th Wkt','8th Wkt','9th Wkt','10th Wkt',
                                                   '1st balls','2nd balls','3rd balls','4th balls','5th balls',
                                          '6th balls','7th balls','8th balls','9th balls','10th balls',
                                           '1st out','2nd out','3rd out','4th out','5th out',
                                          '6th out','7th out','8th out','9th out','10th out'])


    #fow_list = []
    #fow_balls_ = []
    i = -1

    for m in df_sim[(df_sim.batting_team==team)&(df_sim.match_id.str.contains('S02'))]['match_id'].unique():
        #print(m)
        i+=1
        sub_df = df_sim[(df_sim.match_id==m)&(df_sim.wickets_down<=(wkt+1))&(df_sim.batting_team==team)]

        fow = sub_df[sub_df.isWicket==1]['last_fow'].astype(int).tolist()
        fow_rr = sub_df[sub_df.isWicket==1]['last_fow_run_rate'].tolist()
        fow_balls = sub_df[sub_df.isWicket==1]['legal_balls_bowled'].astype(int).tolist()
        #list(int(i) for i in 6*pd.Series(fow)/pd.Series(fow_rr))
        w = [1] * len(fow)

        #### fetch last wkt stuff
        if (sub_df['legal_balls_bowled'].max()-max(fow_balls))>0:
            lwr = sub_df['runs_scored'].max()#-max(fow)
            lwb = sub_df['legal_balls_bowled'].max()#-max(fow_balls)

            fow.append(lwr)
            fow_balls.append(lwb)
            w.append(0)

        #fow_list.append(fow)
        #fow_balls_.append(fow_balls)
        #wkts.
        #print(m, fow, fow_balls, w)

        partnership_df.loc[i,'team'] = team
        partnership_df.loc[i,'Match'] = m
        partnership_df.iloc[i, range(2,len(fow)+2)] = fow
        partnership_df.iloc[i, range(12,len(fow_balls)+12)] = fow_balls
        partnership_df.iloc[i, range(22,len(w)+22)] = w


    
    
    

    for i in range(11, 2, -1):
        a = partnership_df.columns[i-1]
        b = partnership_df.columns[i]
        #print(a,b)
        partnership_df[b] = \
            partnership_df.apply(lambda row: safe_subtract(row[a], row[b]), axis=1)

    for i in range(21, 12, -1):
        a = partnership_df.columns[i-1]
        b = partnership_df.columns[i]
        #print(a,b)
        partnership_df[b] = \
            partnership_df.apply(lambda row: safe_subtract(row[a], row[b]), axis=1)




    wkt_list = ['1st','2nd','3rd',4,5,6,7,8,9,10]
    runs_ = []
    balls_ = []
    outs_ = []

    for w in wkt_list:
        collist = partnership_df.columns[partnership_df.columns.str.startswith(str(w))]
        #print(collist)
        sub_ = partnership_df[collist]
        run_col = collist[0]
        ball_col = collist[1]
        out_col = collist[2]
        runs = sub_[run_col].dropna().values
        balls = sub_[ball_col].dropna().values
        out = sub_[out_col].dropna().values
        #print(runs,balls, out)
        runs_.append(runs)
        balls_.append(balls)
        outs_.append(out)

    avg_runs = []
    avg_balls = []
    sum_runs = []
    outs = []

    for i in range(1,11):
        avg_runs.append(np.mean(runs_[i-1]))
        avg_balls.append(np.mean(balls_[i-1]))
        sum_runs.append(np.sum(runs_[i-1]))
        outs.append(np.sum(outs_[i-1]))

    p_df = pd.DataFrame([avg_runs, avg_balls,sum_runs, outs]).round(2)
    p_df = p_df.append(p_df.iloc[2,:]/p_df.iloc[3,:], ignore_index=True)

    p_df = p_df.T

    p_df['wkt_info'] = p_df.apply(
            lambda x: f"{x[0]}\n SR={np.round(100 * x[0] / x[1], 2)}, Avg={np.round(x[4],2)}",
            axis=1
        )

    #p_df = p_df['wkt_info']



    #p_df_mod = pd.DataFrame()
    print(team)

    #p_df_mod['wkt_info'] = p_df['wkt_info']
    
    p_df = p_df.T
    p_df.columns = ['1st Wkt','2nd Wkt','3rd Wkt','4th Wkt','5th Wkt',
                                          '6th Wkt','7th Wkt','8th Wkt','9th Wkt','10th Wkt']
    p_df['team'] = team

    p_df_all = pd.concat([p_df_all,p_df], axis=0, ignore_index=True)

CSK
DC
GT
LSG
KKR
MI
PBKS
RCB
RR
SRH


/Users/roumyadas/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/roumyadas/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [95]:
p_df_all = p_df_all.iloc[6*np.arange(10)+5, :]

## phase-bowling stats of bowlers (min 30 balls)

In [46]:
bowlers = df_sim[df_sim.season==2].bowler.unique().tolist()
bowl_pattern = '|'.join(bowlers)

bowl_dict = df_sim[df_sim.season == 2][['bowler', 'bowling_team']].drop_duplicates().set_index('bowler')['bowling_team'].to_dict()

def team_fetch(b):
    return bowl_dict[b]

In [47]:
## phase-wise bowling stats

bowl_list = bowl_pattern

phase_ball = df_sim[df_sim.season==2][df_sim.bowler.str.contains(bowl_list)].groupby(['bowler','phase'])\
                        [['runs_conceeded','islegal','isBowlerWicket']].sum().reset_index()

phase_ball['eco'] = 6*phase_ball['runs_conceeded']/phase_ball['islegal']
phase_ball['SR'] = phase_ball['islegal']/phase_ball['isBowlerWicket']

phase_ball['team'] = phase_ball['bowler'].apply(team_fetch)

phase_ball = phase_ball.rename(columns = {'islegal':'balls','runs_conceeded':'runs_conceded',
                                          'isBowlerWicket':'wkts'})

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/1490662040.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  phase_ball = df_sim[df_sim.season==2][df_sim.bowler.str.contains(bowl_list)].groupby(['bowler','phase'])\


In [48]:
pp_bowlers = phase_ball[(phase_ball.phase=='PP')&(phase_ball.balls>=30)].round(2)\
            .sort_values('wkts', ascending=False)\
                .reset_index(drop=True)
                #.head(15)
    
mid_bowlers = phase_ball[(phase_ball.phase=='Middle')&(phase_ball.balls>=30)].round(2)\
            .sort_values('wkts', ascending=False)\
                .reset_index(drop=True)

dt_bowlers = phase_ball[(phase_ball.phase=='Death')&(phase_ball.balls>=30)].round(2)\
            .sort_values('wkts', ascending=False)\
                .reset_index(drop=True)

## phase-wise batter stats (min 30 balls)

In [52]:
batters = df_sim[df_sim.season==2].striker.unique().tolist()
bat_pattern = '|'.join(batters)
bat_dict = df_sim[df_sim.season == 2][['striker', 'batting_team']].drop_duplicates().set_index('striker')['batting_team'].to_dict()

def team_fetch(b):
    return bat_dict[b]

In [53]:
phase_bat = df_sim[df_sim.season==2].groupby(['striker','phase'])[['runs_off_bat','is_faced_by_batter']].sum().reset_index()
out_df = df_sim[df_sim.season==2][df_sim.isWicket==1].groupby(['player_dismissed','phase'])['isWicket'].sum().reset_index()

out_df.columns= ['striker','phase','outs']
phase_bat = phase_bat.merge(out_df, on=['striker','phase'], how='outer')

phase_bat = phase_bat.rename(columns = {'runs_off_bat':'runs',
                                        'is_faced_by_batter':'balls'})

phase_bat['avg'] = phase_bat['runs']/phase_bat['outs']
phase_bat['SR'] = 100*phase_bat['runs']/phase_bat['balls']

phase_bat['team'] = phase_bat['striker'].apply(team_fetch)

phase_bat['runs'] = phase_bat['runs'].fillna(0).astype(int)
phase_bat['balls'] = phase_bat['balls'].fillna(0).astype(int)
phase_bat['outs'] = phase_bat['outs'].fillna(0).astype(int)
phase_bat = phase_bat.round(2)


/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/928469949.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  out_df = df_sim[df_sim.season==2][df_sim.isWicket==1].groupby(['player_dismissed','phase'])['isWicket'].sum().reset_index()


In [54]:
pp_batters = phase_bat[(phase_bat.phase=='PP')&(phase_bat.balls>=30)].round(2)\
            .sort_values('runs', ascending=False)\
                .reset_index(drop=True)
                #.head(15)
    
mid_batters = phase_bat[(phase_bat.phase=='Middle')&(phase_bat.balls>=30)].round(2)\
            .sort_values('runs', ascending=False)\
                .reset_index(drop=True)

dt_batters = phase_bat[(phase_bat.phase=='Death')&(phase_bat.balls>=30)].round(2)\
            .sort_values('runs', ascending=False)\
                .reset_index(drop=True)

## pitch-wise bowler-stats (min 30 balls)

In [76]:
bowlers = df_sim[df_sim.season==2].bowler.unique().tolist()
bowl_pattern = '|'.join(bowlers)

bowl_dict = df_sim[df_sim.season == 2][['bowler', 'bowling_team']].drop_duplicates().set_index('bowler')['bowling_team'].to_dict()

def team_fetch(b):
    return bowl_dict[b]

In [77]:
## pitch-wise bowling stats

bowl_list = bowl_pattern

pitch_ball = df_sim[df_sim.season==2][df_sim.bowler.str.contains(bowl_list)].groupby(['bowler','pitch'])\
                        [['runs_conceeded','islegal','isBowlerWicket']].sum().reset_index()

pitch_ball['eco'] = 6*pitch_ball['runs_conceeded']/pitch_ball['islegal']
pitch_ball['SR'] = pitch_ball['islegal']/pitch_ball['isBowlerWicket']

pitch_ball['team'] = pitch_ball['bowler'].apply(team_fetch)

pitch_ball = pitch_ball.rename(columns = {'islegal':'balls','runs_conceeded':'runs_conceded',
                                          'isBowlerWicket':'wkts'})

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/1140617325.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pitch_ball = df_sim[df_sim.season==2][df_sim.bowler.str.contains(bowl_list)].groupby(['bowler','pitch'])\


In [81]:
spin_bowlers = pitch_ball[(pitch_ball.pitch=='spin')&(pitch_ball.balls>=30)].round(2)\
            .sort_values('wkts', ascending=False)\
                .reset_index(drop=True)
                #.head(15)
    
neutralish_bowlers = pitch_ball[(pitch_ball.pitch=='neutralish')&(pitch_ball.balls>=30)].round(2)\
            .sort_values('wkts', ascending=False)\
                .reset_index(drop=True)

pace_bowlers = pitch_ball[(pitch_ball.pitch=='pace')&(pitch_ball.balls>=30)].round(2)\
            .sort_values('wkts', ascending=False)\
                .reset_index(drop=True)

## pitch-wise batter stats

In [85]:
batters = df_sim[df_sim.season==2].striker.unique().tolist()
bat_pattern = '|'.join(batters)
bat_dict = df_sim[df_sim.season == 2][['striker', 'batting_team']].drop_duplicates().set_index('striker')['batting_team'].to_dict()

def team_fetch(b):
    return bat_dict[b]

In [86]:
pitch_bat = df_sim[df_sim.season==2].groupby(['striker','pitch'])[['runs_off_bat','is_faced_by_batter']].sum().reset_index()
out_df = df_sim[df_sim.season==2][df_sim.isWicket==1].groupby(['player_dismissed','pitch'])['isWicket'].sum().reset_index()

out_df.columns= ['striker','pitch','outs']
pitch_bat = pitch_bat.merge(out_df, on=['striker','pitch'], how='outer')

pitch_bat = pitch_bat.rename(columns = {'runs_off_bat':'runs',
                                        'is_faced_by_batter':'balls'})

pitch_bat['avg'] = pitch_bat['runs']/pitch_bat['outs']
pitch_bat['SR'] = 100*pitch_bat['runs']/pitch_bat['balls']

pitch_bat['team'] = pitch_bat['striker'].apply(team_fetch)

pitch_bat['runs'] = pitch_bat['runs'].fillna(0).astype(int)
pitch_bat['balls'] = pitch_bat['balls'].fillna(0).astype(int)
pitch_bat['outs'] = pitch_bat['outs'].fillna(0).astype(int)
pitch_bat = pitch_bat.round(2)


/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/385777234.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  out_df = df_sim[df_sim.season==2][df_sim.isWicket==1].groupby(['player_dismissed','pitch'])['isWicket'].sum().reset_index()


In [87]:
spin_batters = pitch_bat[(pitch_bat.pitch=='spin')&(pitch_bat.balls>=30)].round(2)\
            .sort_values('runs', ascending=False)\
                .reset_index(drop=True)
                #.head(15)
    
neutralish_batters = pitch_bat[(pitch_bat.pitch=='neutralish')&(pitch_bat.balls>=30)].round(2)\
            .sort_values('runs', ascending=False)\
                .reset_index(drop=True)

pace_batters = pitch_bat[(pitch_bat.pitch=='pace')&(pitch_bat.balls>=30)].round(2)\
            .sort_values('runs', ascending=False)\
                .reset_index(drop=True)

## ground-wise batter stats (min 100 runs)

In [57]:
df_sim.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'wicket_type', 'player_dismissed', 'legal_balls_bowled', 'wicket_event',
       'fielder', 'legality', 'bowler_wicket', 'islegal', 'isDotforBatter',
       'isOne', 'isTwo', 'isThree', 'isStrikeRotation', 'isFour', 'isSix',
       'isBowlerWicket', 'is_striker_Out', 'is_nonstriker_Out', 'isWicket',
       'total_runs', 'runs_scored', 'wickets_down', 'run_rate', 'last_fow',
       'partnership_runs', 'isno', 'is_faced_by_batter', 'last_fow_run_rate',
       'fours', 'sixes', 'striker_batting_position',
       'non_striker_batting_position', 'runs_conceeded', 'isDotforbowler',
       'legal_balls_remaining', 'target', 'runs_remaining', 'reqd_run_rate',
       'over', 'phase', 'before_ball_partnership', 'after_ball_partnership',
       'before_balls', 'after_balls', '

In [159]:
g_run = pd.pivot_table(data=df_sim[df_sim.season==2], values='runs_off_bat',
               index=['striker'], columns='venue', aggfunc='sum').reset_index()

g_run['total'] = g_run[g_run.columns[1:]].sum(axis=1)
g_run = g_run[g_run.total>=100].sort_values('total', ascending=False).reset_index(drop=True)

g_ball = pd.pivot_table(data=df_sim[df_sim.season==2], values='is_faced_by_batter',
               index=['striker'], columns='venue', aggfunc='sum').reset_index()

g_ball['total'] = g_ball[g_ball.columns[1:]].sum(axis=1)

for i,row in g_run.iterrows():
    cols = g_run.columns[1:]
    bat = row['striker']
    for col in cols:
        if pd.isna(row[col])==False:
            runs = row[col]
            balls = g_ball[g_ball.striker==bat][col].unique()[0]
            score = f"{int(runs)} off {int(balls)}, \n SR = {np.round(100*runs/balls,2)}"
            g_run.loc[i, col] = score

In [160]:
g_run_bat= g_run

In [161]:
g_wkt = pd.pivot_table(data=df_sim[df_sim.season==2], values='isBowlerWicket',
               index=['bowler'], columns='venue', aggfunc='sum').reset_index()

g_wkt['total'] = g_wkt[g_wkt.columns[1:]].sum(axis=1)
g_wkt = g_wkt[g_wkt.total>=2].sort_values('total', ascending=False).reset_index(drop=True)

g_ball = pd.pivot_table(data=df_sim[df_sim.season==2], values='islegal',
               index=['bowler'], columns='venue', aggfunc='sum').reset_index()

g_ball['total'] = g_ball[g_ball.columns[1:]].sum(axis=1)

g_run = pd.pivot_table(data=df_sim[df_sim.season==2], values='runs_conceeded',
               index=['bowler'], columns='venue', aggfunc='sum').reset_index()

g_run['total'] = g_run[g_run.columns[1:]].sum(axis=1)

for i,row in g_wkt.iterrows():
    cols = g_wkt.columns[1:]
    bowl = row['bowler']
    for col in cols:
        if pd.isna(row[col])==False:
            wkts = row[col]
            balls = g_ball[g_ball.bowler==bowl][col].unique()[0]
            runs = g_run[g_run.bowler==bowl][col].unique()[0]
            
            score = f"{int(wkts)} wkt(s), Eco = {np.round(6*runs/balls,2)}, \n SR = {np.round(balls/wkts,2)}"
            g_wkt.loc[i, col] = score

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/2506825665.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  score = f"{int(wkts)} wkt(s), Eco = {np.round(6*runs/balls,2)}, \n SR = {np.round(balls/wkts,2)}"


g_wkt

## venue-wise 1st inning avg score

In [143]:
v_scores = df_sim[(df_sim.season==2)&(df_sim.innings==1)].groupby(['venue','match_id','batting_team'])\
                            [['runs_scored','wickets_down']].max().reset_index()

v_avg_score = v_scores.groupby('venue')[['runs_scored','wickets_down']].mean().reset_index()

v_avg_score = v_avg_score.round(1)

v_avg_score['total'] = v_avg_score[['runs_scored','wickets_down']].apply(lambda x: f"{x[0]} - {x[1]}", axis=1)

v_avg_score = v_avg_score[['venue','total']]

v_totals = df_sim[(df_sim.season==2)].groupby(['venue','match_id','batting_team'])\
                            [['runs_scored','wickets_down']].max().reset_index()

v_wins = pd.pivot_table(data=df_sim[df_sim.season==2], values='runs_scored',
               index=['venue','match_id'], columns='innings', aggfunc='max').reset_index()

v_wins.columns = ['venue','match_id','bat_1st','bat_2nd']

v_wins['winner'] = v_wins[['bat_1st','bat_2nd']].apply(lambda x: 'bat 1st' if x[0]>x[1] else 'bat 2nd', axis=1)

#pd.pivot_table(data=v_wins, values='winner',
#               index=['venue','winner'], columns='winner', aggfunc='count').reset_index()
v_wins = v_wins.groupby(['venue','winner'])['bat_1st'].count().reset_index()

v_avg_score['bat_1st_wins'] = 0
v_avg_score['bat_2nd_wins'] = 0

for index,row in v_avg_score.iterrows():
    v = row['venue']
    b1w = v_wins[v_wins.venue==v][v_wins.winner=='bat 1st']['bat_1st'].unique()[0]
    b2w = v_wins[v_wins.venue==v][v_wins.winner=='bat 2nd']['bat_1st'].unique()[0]
    
    v_avg_score.loc[index, 'bat_1st_wins'] = b1w
    v_avg_score.loc[index, 'bat_2nd_wins'] = b2w
    
v_avg_score

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/1963320737.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  b1w = v_wins[v_wins.venue==v][v_wins.winner=='bat 1st']['bat_1st'].unique()[0]
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/1963320737.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  b2w = v_wins[v_wins.venue==v][v_wins.winner=='bat 2nd']['bat_1st'].unique()[0]


,venue,total,bat_1st_wins,bat_2nd_wins
0,CSK,184.7 - 5.3,5,4
1,DC,204.6 - 5.0,3,4
2,GT,186.6 - 5.3,1,6
3,KKR,186.3 - 6.3,3,4
4,LSG,186.0 - 5.4,5,2
5,MI,189.3 - 4.9,3,4
6,PBKS,178.7 - 5.7,4,5
7,RCB,182.3 - 5.3,3,4
8,RR,177.7 - 6.7,2,5
9,SRH,200.0 - 3.0,6,1


## breakthrough-bowlers (partnership>=40)

In [154]:
bb = df_sim[df_sim.season==2][df_sim.partnership_runs>=40].groupby('bowler')\
            [['runs_conceeded','islegal','isBowlerWicket']].sum().reset_index()

bb.columns = ['bowler','runs_conceded','balls','wkts']
bb = bb[bb.wkts>=1]
bb['Eco'] = 6*bb['runs_conceded']/bb['balls']
bb['SR'] = bb['balls']/bb['wkts']
bb = bb.sort_values(['wkts','SR'], ascending=[False,True])

bb = bb.round(2)


/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/295832551.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bb = df_sim[df_sim.season==2][df_sim.partnership_runs>=40].groupby('bowler')\


In [157]:
df_sim[df_sim.season==2][df_sim.phase=='Death'].groupby(['match_id','innings'])\
[['total_runs','is_faced_by_batter','isWicket']].sum().reset_index().sort_values('total_runs',ascending=False)

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_3351/459365417.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_sim[df_sim.season==2][df_sim.phase=='Death'].groupby(['match_id','innings'])\


,match_id,innings,total_runs,is_faced_by_batter,isWicket
104,S02M054,1,78,30,1
133,S02M068,2,71,30,3
77,S02M040,1,70,30,1
59,S02M031,1,69,30,2
142,S02M073,1,69,30,1
88,S02M046,1,68,30,1
8,S02M005,1,67,30,2
114,S02M059,1,65,30,2
120,S02M062,1,65,30,1
143,S02M073,2,65,30,0


rest:::
            
1. figure out some specifics of bowlers (under pressure, breakthrough ability, etc.)
2. gather some similar stats of batters
3. create a stats file on avg score batting 1st on grounds, pitch-wise
4. win-loss record on ground based on toss, pitch
5. f2f record of teams

# saving

In [165]:
##saving all STATS in one excel##
excel_filename = "/Users/roumyadas/Desktop/IPL_Simulation/Season_02/stats ppts/SPL_S02_special_stats.xlsx"

# Use ExcelWriter to write multiple sheets
with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
    p_df_all.to_excel(writer, sheet_name='wkt-wise-partnership', index=False)
    bb.to_excel(writer, sheet_name='Breakthrough-Bowlers', index=False)
    #---------#
    pp_bowlers.to_excel(writer, sheet_name='PP_bowlers', index=False)
    mid_bowlers.to_excel(writer, sheet_name='Middle_bowlers', index=False)
    dt_bowlers.to_excel(writer, sheet_name='Death_bowlers', index=False)
    #---------#
    pp_batters.to_excel(writer, sheet_name='PP_batters', index=False)
    mid_batters.to_excel(writer, sheet_name='Middle_batters', index=False)
    dt_batters.to_excel(writer, sheet_name='Death_batters', index=False)
    #---------#
    spin_bowlers.to_excel(writer, sheet_name='Spin_pitch_bowlers', index=False)
    neutralish_bowlers.to_excel(writer, sheet_name='Neutral_pitch_bowlers', index=False)
    pace_bowlers.to_excel(writer, sheet_name='Pace_pitch_bowlers', index=False)
    #---------#
    spin_batters.to_excel(writer, sheet_name='Spin_pitch_batters', index=False)
    neutralish_batters.to_excel(writer, sheet_name='Neutral_pitch_batters', index=False)
    pace_batters.to_excel(writer, sheet_name='Pace_pitch_batters', index=False)
    #---------#
    g_run_bat.to_excel(writer, sheet_name='Ground-wise-batting_stats', index=False)
    g_wkt.to_excel(writer, sheet_name='Ground-wise-bowling_stats', index=False)
    v_avg_score.to_excel(writer, sheet_name='Ground-wise-avg-score-and-wins', index=False)
    #---------#
    
    #---------#
    
    
    
print(f"Excel file '{excel_filename}' created successfully with 17 sheets!")

Excel file '/Users/roumyadas/Desktop/IPL_Simulation/Season_02/stats ppts/SPL_S02_special_stats.xlsx' created successfully with 17 sheets!


In [166]:

# Dictionary mapping sheet names to DataFrames
sheets_data = {
    'wkt-wise-partnership': p_df_all,
    'Breakthrough-Bowlers': bb,
    'PP_bowlers':pp_bowlers,
    'Middle_bowlers':mid_bowlers,
    'Death_bowlers':dt_bowlers,
    'PP_batters':pp_batters,
    'Middle_batters':mid_batters,
    'Death_batters':dt_batters,
    'Spin_pitch_bowlers':spin_bowlers,
    'Neutral_pitch_bowlers':neutralish_bowlers,
    'Pace_pitch_bowlers':pace_bowlers,
    'Spin_pitch_batters':spin_batters,
    'Neutral_pitch_batters':neutralish_batters,
    'Pace_pitch_batters':pace_batters,
    'Ground-wise-batting_stats':g_run_bat,
    'Ground-wise-bowling_stats':g_wkt,
    'Ground-wise-avg-score-and-wins':v_avg_score
        
    
}

with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
    # Writing all sheets
    for sheet_name, df in sheets_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Get workbook
    workbook = writer.book  

    # Define formats
    bold_format = workbook.add_format({'bold': True})  # Bold text
    large_font_format = workbook.add_format({'font_size': 14})  # Increased font size
    red_text_format = workbook.add_format({'font_color': 'red'})  # Red text

    # Apply formatting to each sheet
    for sheet_name, df in sheets_data.items():
        worksheet = writer.sheets[sheet_name]
        
        # 1️⃣ Bold all column names (header row)
        for col_idx, col_name in enumerate(df.columns):
            worksheet.write(0, col_idx, col_name, bold_format)
        
        # 3️⃣ Bold entire column 1 (index 0) of every sheet
        worksheet.set_column(0, 0, None, bold_format)

            
        # 6️⃣ Adjust column widths dynamically
        for col_idx, col_name in enumerate(df.columns):
            max_length = max(df[col_name].astype(str).apply(len).max(), len(col_name))  # Max of column values & header
            worksheet.set_column(col_idx, col_idx, max_length + 2)  # Adding padding for readability
            
        # 7️⃣ Freeze the first row of every sheet
        worksheet.freeze_panes(1, 0)
        
        # 8️⃣ **Add Filter to all columns**
        worksheet.autofilter(0, 0, 0, len(df.columns) - 1)  # Apply filter to the first row across all columns


print(f"Excel file '{excel_filename}' created successfully with formatting!")


Excel file '/Users/roumyadas/Desktop/IPL_Simulation/Season_02/stats ppts/SPL_S02_special_stats.xlsx' created successfully with formatting!
